## 1C

> Let us assume the plaintext is made of 26 capital letters only. So, the $n=26$. Given the affine cipher $c = 5p + 9 \mod 26$, what is the ciphertext for the plaintext “CRYPTO IS FUN”. Here we keep space as is

The ciphertext is `TQZGAB XV IFW`

In [31]:
import string
from typing import List, Tuple

Coefficients = Tuple[int, int]
Keyspace = List[str]

def check_assertions(n: int, a: int, b: int):
    if a < 0 or b < 0 or a >= n or b >= n:
        raise ValueError(f'a and b must be non-negative and less than {n}')

def encrypt(plaintext: str, coefficients: Coefficients, keyspace: Keyspace) -> str:
    n = len(keyspace)
    a, b = coefficients
    check_assertions(n, a, b)

    plaintext_words: List[str] = plaintext.split(' ')
    ciphertext_words: List[str] = []

    for word in plaintext_words:
        indicies = [ keyspace.index(p) for p in word ]
        ciphertext_words.append(
            ''.join(keyspace[(a * idx + b) % n] for idx in indicies)
        )

    return ' '.join(ciphertext_words)

def decrypt(ciphertext: str, coefficients: Coefficients, keyspace: Keyspace) -> str:
    n = len(keyspace)
    a, b = coefficients
    check_assertions(n, a, b)
    
    def inverse(a, n) -> int:
        import math

        possible = [ a for a in range(n) if math.gcd(a, n) == 1]
        for p in possible:
            if (a * p) % n == 1:
                return p
            
        raise ValueError(f'No inverse for {a} mod {n}')
    
    ciphertext_words: List[str] = ciphertext.split(' ')
    plaintext_words: List[str] = []
    a_inverse = inverse(a, n)

    for word in ciphertext_words:
        indicies = [ keyspace.index(p) for p in word ]
        plaintext_words.append(
            ''.join(keyspace[(a_inverse * (idx - b)) % n] for idx in indicies)
        )

    return ' '.join(plaintext_words)
    

coefficients: Coefficients = (5, 9)
keyspace: Keyspace = list(string.ascii_uppercase)

c = encrypt("CRYPTO IS FUN", coefficients, keyspace)
assert(c == "TQZGAB XV IFW")
assert("CRYPTO IS FUN" == decrypt(c, coefficients, keyspace))

## 1D

> Eve has the ciphertext “QJKES REOGH GXXRE OXEO”. She magically knows the cipher is an affine cipher and the letter T is encrypted to H and O to E. Recover the decryption function and decipher the message. Students shall solve it manually first and then use code to solve it. They both shall give the same results. The code shall be more general, not just in this case.

In [37]:
ciphertext = "QJKES REOGH GXXRE OXEO"
O_indicies = [idx for idx, c in enumerate(ciphertext) if c == "O"]
H_indicies = [idx for idx, c in enumerate(ciphertext) if c == "H"]

for a in range(len(keyspace)):
    for b in range(len(keyspace)):
        try:
            plaintext = decrypt(ciphertext, (a, b), keyspace)
            assert(all(
                [plaintext[idx] == "E" for idx in O_indicies]
            ))
            assert(all(
                [plaintext[idx] == "T" for idx in H_indicies]
            ))

            print(a, b, plaintext)

        except (AssertionError, ValueError) as e:
            pass

assert(
    "WLUSO FSEKT KHHFS EHSE" == decrypt(ciphertext, (3, 2), keyspace)
)

3 2 WLUSO FSEKT KHHFS EHSE


## 1E

> What is the affine formula if we want to include the space and little letter case in the encde set?

$$
n = \lvert \{A, \dots, Z\} \cup \{a, \dots, z\} \cup \{ \text{ } \} \rvert = 53 \\
E = (ap + b)\mod{n} \\
D = a^{-1}(p - b)\mod{n}
$$